In [1]:
import requests
from bs4 import BeautifulSoup
import os
import polars as pl
from zipfile import ZipFile
from datetime import datetime

import utils.source
from utils.request_market_data import (
    request_prices_to_landing,
    write_prices_to_raw,
    read_prices_from_lake
)

from utils.request_cvm_ciasabertas import (
    download_files,
    read_files_from_landing,
    write_files_to_lake,
    read_files_from_lake,
    trasnform_trusted_accounts,
    trasnform_trusted_tickers
)


In [3]:
DOCUMENT = 'fca'
REPORTS = ['valor_mobiliario']

for REPORT in REPORTS:
    DIR = DOCUMENT+'/'+REPORT
    df = read_files_from_landing(DOCUMENT, REPORT)
    write_files_to_lake(df, 'raw', DIR)
    df = read_files_from_lake('raw', DIR)
    df = trasnform_trusted_tickers(df)
    write_files_to_lake(df, 'trusted', DIR)